In [49]:
from fuzzywuzzy import fuzz
import pandas as pd
from numpy import triu, ones
import numpy as np

In [50]:
with open("product_list2.csv", "r") as fp:
    data_tgt = fp.readlines()

df = pd.read_csv("scraped list.csv")
    
list_tgt = [x.replace('"',"") for x in list(data_tgt)]
list_src = df['product-name'].values.tolist()#[:500]

In [51]:
def clean(s):
    d = {}
    s = s.lower()
    sl = s.split(" ")
    d.update({'brand': sl[0], 'type': sl[1]})
    second = sl[2:]
    for part in second:
        if "mb/s" in part:
            d.update({'speed':part})
        if "gb" in part:
            d.update({'capacity':part[0:-1].replace('\n', "")})
            
    return d

In [52]:
prodict = {}
for l in list_tgt:
    prodict.update({l: clean(l)})

In [53]:
df = pd.DataFrame(prodict).T
df.head()

,brand,capacity,speed,type
KINGSTON CF CARD 4GB\n,kingston,4gb,NaN,cf
KINGSTON CF CARD 8GB\n,kingston,8gb,NaN,cf
KINGSTON SDC CLASS 10 80MB/S 128GB\n,kingston,128gb,80mb/s,sdc
KINGSTON SDC CLASS 10 80MB/S 16GB\n,kingston,16gb,80mb/s,sdc
KINGSTON SDC CLASS 10 80MB/S 256MB\n,kingston,NaN,80mb/s,sdc


In [54]:
df.isnull().sum()

brand        0
capacity     1
speed       19
type         0
dtype: int64

In [55]:
df['num'] = df.notnull().sum(axis=1)

In [56]:
df.head()

,brand,capacity,speed,type,num
KINGSTON CF CARD 4GB\n,kingston,4gb,NaN,cf,3
KINGSTON CF CARD 8GB\n,kingston,8gb,NaN,cf,3
KINGSTON SDC CLASS 10 80MB/S 128GB\n,kingston,128gb,80mb/s,sdc,4
KINGSTON SDC CLASS 10 80MB/S 16GB\n,kingston,16gb,80mb/s,sdc,4
KINGSTON SDC CLASS 10 80MB/S 256MB\n,kingston,NaN,80mb/s,sdc,3


In [57]:
type(df.iloc[0,0])

str

In [64]:
l = list_src[5]
print(l)
df_temp = df.drop(['num'], axis=1).applymap(lambda x: 1 if type(x) == str and x in l.lower() else 0)
key = df_temp.sum(axis=1).sort_values(ascending=False).index.tolist()[0]
num = df_temp.sum(axis=1).sort_values(ascending=False).values.tolist()[0]
print(key,num)
df['num'].loc[key]

Samsung Evo Plus Micro Sdxc Class 10 Uhs 1 64G Memory Card
SAMSUNG SDC EVO CLASS 10 100MB/S 128GB
 1


4

In [59]:
matchdict = {}
for s in list_src:
    df_temp = df.drop(['num'], axis=1).applymap(lambda x: 1 if type(x) == str and x in s.lower() else 0)
    key = df_temp.sum(axis=1).sort_values(ascending=False).index.tolist()[0]
    num = df_temp.sum(axis=1).sort_values(ascending=False).values.tolist()[0]
    if df['num'].loc[key] == num:
        matchdict.setdefault(key, []).append(s)

In [72]:
i = 0
tot = 0
for key, val in matchdict.items():
    print(i)
    print(key)
    tot += len(val)
    for v in val:
        print(v)
    print('-'*10)
    i += 1
print(i, tot)

0
SANDISK SDHC CARDS CLASS 10 - ULTRA - 80MB/S 32GB

Sandisk Ultra 32Gb Sdhc Uhs I U1 (Up To 80Mb/S Read) Memory Card
Sandisk Ultra 32Gb Microsdhc Ush 1 80Mb/S With Adapter
Sandisk Ultra 32Gb Sdhc Class10   80Mb/S
Sandisk Ultra 32Gb Microsdhc Uhs I (Up To 80Mb/S Read) Memory Cardw/Adapter
----------
1
SANDISK CF EXTREME 120MB/S SERIES 16GB

Sandisk 16Gb Cf Extreme (120Mb/S) (Export)
----------
2
SANDISK CF EXTREME PRO 160MB/S SERIES 64GB

Sandisk Extreme Pro 64Gb Cf Card 160Mb/S (Sdcfxps 064G X46)
Sandisk 64Gb Cf Extreme Pro (160Mb/S) (Export)
Sandisk Extreme Pro 160Mb/S 64Gb Cf   Black (Export)
----------
3
SANDISK CF EXTREME PRO 160MB/S SERIES 128GB

Sandisk Extreme Pro 128Gb Cf Card 160Mb/S (Sdcfxps 128G X46)
Sandisk Extreme Pro 160Mb/S Cf 128Gb   Black (Export)
Sandisk Extreme Pro 128Gb Cf Card 160Mb/S
----------
4
SANDISK SDHC CARDS CLASS 10 - ULTRA - 80MB/S 16GB

Sandisk Ultra 16Gb Microsdhc Uhs I Class 10 Card 80Mb/S With Sdadapter
Sandisk 16Gb 16G Ultra Sd Sdhc Full Hd Video Me